In [242]:
import import_ipynb
import openai
from dotenv import load_dotenv
import requests
import re
from bs4 import BeautifulSoup
import os
from datetime import datetime, timedelta, timezone
import locale
import unicodedata

In [243]:
%run "../jgc_news/g1.ipynb" import G1
%run "../jgc_news/dn.ipynb" import DN
%run "../jgc_news/odia.ipynb" import ODia
%run "../jgc_news/estadao.ipynb" import Estadao
%run "../jgc_news/folha.ipynb" import Folha
%run "../jgc_news/espn.ipynb" import Espn
%run "../jgc_news/uol.ipynb" import Uol
%run "../jgc_news/correio_braziliense.ipynb" import CorreioBraziliense

load_dotenv()
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [244]:
def str_cleasing(str):
    str = re.sub(r'\n', ' ', str)
    str = re.sub(r'\s+', ' ', str)
    str = re.sub(r'\s+$', ' ', str)
    str = re.sub(r'^\s+', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    return str

In [245]:
def normalize_string(text):
    normalized_text = unicodedata.normalize('NFKD', text)
    text_without_accents = ''.join([c for c in normalized_text if not unicodedata.combining(c)])
    final_text = text_without_accents.lower().replace(' ', '-')
    return final_text

In [246]:
teams = [
    {
        "teamName": 'Athletico-PR', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pr/futebol/times/athletico-pr/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/atletico-paranaense/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/atletico-pr/"}
        ]}, {
        "teamName": 'Atlético-GO', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/go/futebol/times/atletico-go/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/atletico-clube-goianiense/"}
        ]},{
        "teamName": 'Atlético-MG', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/atletico-mg/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/atletico-mineiro/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/atletico-mg/"}
        ]},{
        "teamName": 'Bahia', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/ba/futebol/times/bahia/"}
        ]},{
        "teamName": 'Botafogo', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/botafogo/"},
            {"source": "odia", "url": "https://odia.ig.com.br/esporte/botafogo"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/botafogo/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/botafogo/"}
        ]},{
        "teamName": 'Bragantino', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/vale-do-paraiba-regiao/futebol/times/bragantino/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/bragantino/"}
        ]},{
        "teamName": 'Corinthians', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/corinthians/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/corinthians/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/corinthians/"}
        ]},{
        "teamName": 'Criciúma', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sc/futebol/times/criciuma/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/criciuma-esporte-clube/"}
        ]},{
        "teamName": 'Cruzeiro', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/cruzeiro/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/cruzeiro-esporte-clube/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/cruzeiro/"}
        ]},{
        "teamName": 'Cuiaba', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/mt/futebol/times/cuiaba/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/cuiaba-esporte-clube/"}
        ]},{
        "teamName": 'Flamengo', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/flamengo/"},
            {"source": "odia", "url": "https://odia.ig.com.br/esporte/flamengo"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/flamengo/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/flamengo/"}
        ]},{
        "teamName": 'Fluminense', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/fluminense/"},
            {"source": "odia", "url": "https://odia.ig.com.br/esporte/fluminense"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/fluminense/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/fluminense/"}
        ]},
        {
        "teamName": 'Fortaleza', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/ce/futebol/times/fortaleza/"},
            {"source": "dn", "url": "https://diariodonordeste.verdesmares.com.br/noticias/sobre/Esportes-fortaleza%20esporte%20clube"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/https://www.estadao.com.br/tudo-sobre/fortaleza-esporte-clube/"}
        ]}
        ,{
        "teamName": 'Grêmio', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/rs/futebol/times/gremio/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/gremio/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/gremio/"}
        ]},{
        "teamName": 'Internacional', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/rs/futebol/times/internacional/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/internacional/"}
        ]},{
        "teamName": 'Juventude', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/rs/futebol/times/juventude/"}
        ]},{
        "teamName": 'Palmeiras', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/palmeiras/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/palmeiras/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/palmeiras/"}
        ]},{
        "teamName": 'São-Paulo', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/sao-paulo/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/sao-paulo-futebol-clube/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/sao-paulo-fc/"}
        ]},{
        "teamName": 'Vasco', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/vasco/"},
            {"source": "odia", "url": "https://odia.ig.com.br/esporte/vasco"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/vasco/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/vasco/"}
        ]},{
        "teamName": 'Vitória', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/ba/futebol/times/vitoria/"}
        ]},{
        "teamName": 'Amazonas-FC', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/am/futebol/times/amazonas-fc/"}
        ]},{
        "teamName": 'América-MG', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/america-mg/"}
        ]},{
        "teamName": 'Avaí', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sc/futebol/times/avai/"}
        ]},{
        "teamName": 'Botafogo-SP', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/ribeirao-preto-e-regiao/futebol/times/botafogo-sp/"}
        ]},{
        "teamName": 'Brusque', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sc/futebol/times/brusque/"}
        ]},{
        "teamName": 'Ceará', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/ce/futebol/times/ceara/"},
            {"source": "dn", "url": "https://diariodonordeste.verdesmares.com.br/noticias/sobre/Esportes-cear%C3%A1%20sporting%20club"}
        ]},{
        "teamName": 'Chapecoense', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sc/futebol/times/chapecoense/"}
        ]},{
        "teamName": 'Coritiba', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pr/futebol/times/coritiba/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/coritiba/"}
        ]},{
        "teamName": 'CRB', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/al/futebol/times/crb/"}
        ]},{
        "teamName": 'Goiás', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/go/futebol/times/goias/"}
        ]},{
        "teamName": 'Guarani', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/campinas-e-regiao/futebol/times/guarani/"}
        ]},{
        "teamName": 'Ituano', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/tem-esporte/futebol/times/ituano/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/ituano/"}
        ]},{
        "teamName": 'Mirassol', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/tem-esporte/futebol/times/mirassol/"}
        ]},{
        "teamName": 'Novorizontino', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/tem-esporte/futebol/times/novorizontino/"}
        ]},{
        "teamName": 'Operário-PR', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pr/futebol/times/operario-pr/"}
        ]},{
        "teamName": 'Paysandu', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pa/futebol/times/paysandu/"}
        ]},{
        "teamName": 'Ponte-Preta', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/campinas-e-regiao/futebol/times/ponte-preta/"}
        ]},{
        "teamName": 'Santos', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/santos-e-regiao/futebol/times/santos/"}
        ]},{
        "teamName": 'Sport', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pe/futebol/times/sport/"}
        ]},{
        "teamName": 'Vila-Nova', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/go/futebol/times/vila-nova/"}
        ]},
    ]

In [247]:
def get_references(teamName, sources):
    newsRefereces = []
    for source in sources:
        if source['source'] == 'g1':
            g1 = G1()
            urls = g1.getUrls(source['url'])
            for url in urls:
                newsData = g1.getNewsData(url)
                newsRefereces.append(newsData)
        elif source['source'] == 'dn':
            dn = DN()
            urls = dn. getUrls(source['url'], normalize_string(teamName))
            for url in urls:
                newsData = dn.getNewsData(url)
                newsRefereces.append(newsData)
        elif source['source'] == 'odia':
            odia = ODia()
            urls = odia. getUrls(source['url'], normalize_string(teamName))
            for url in urls:
                newsData = odia.getNewsData(url)
                newsRefereces.append(newsData)
        elif source['source'] == 'estadao':
            estadao = Estadao()
            urls = estadao. getUrls(source['url'])
            for url in urls:
                newsData = estadao.getNewsData(url)
                newsRefereces.append(newsData)
        elif source['source'] == 'folha':
            folha = Folha()
            urls = folha. getUrls(source['url'])
            for url in urls:
                newsData = folha.getNewsData(url)
                newsRefereces.append(newsData)
        else:
            print('Fonte não encontrada')
    newsRefereces = [x for x in newsRefereces if x is not None]
    
    if len(newsRefereces) == 0:
        return None
    
    reference = ''
    for news in newsRefereces:
        reference += news['title'] + '\n' + news['content'] + '\n\n'
    return reference

In [248]:
def save_sport(team, title, text):
    url = 'http://localhost:3000/api/sport'
    
    now = datetime.now(timezone.utc)
    tomorrow = now + timedelta(days=1)

    payload = {
        "uid": "johannesadmin",
        "supportUid": "johannesadmin",
        "h1": team, 
        "h2": title, 
        "text": text, 
    }
    response = requests.post(url, json=payload)

    if response.status_code != 200:
        print(f'Erro: {response.status_code}')

In [249]:
def splitIntoContent(news):
    newsParagraphs = news.split("\n")
    newsParagraphs = [paragraph for paragraph in newsParagraphs if len(paragraph) > 10]
    title = newsParagraphs[0].replace("*", "").replace("#", "")
    newsParagraphs = newsParagraphs[1:]
    content = "\n\n".join(newsParagraphs)
    return {"title": title, "content": content}

In [250]:
# for team in teams:
#     teamName = team['teamName']
#     sources = team['sources']
#     reference = get_references(teamName, sources)
#     content = f'Escreva um texto jornalístico sem muita enrolação entre 400 e 500 palavras que será publicado em um jornal de grande circulação na seção de esportes falando sobre um time de futebol brasileiro, o {teamName}. Considere como referêcia explusivamente o que foi extraído de alguns sites de prestígio \n\n{reference}\n\nCrie o texto sem enrolação e apenas com informações relevantes sobre o time de futebol {teamName}.'
#     message = [{
#         'role': 'user',
#         'content': content
#     }]
#     response = client.chat.completions.create(
#         messages=message, 
#         model="gpt-4o-mini", 
#         max_tokens=10000, 
#         temperature=0, 
#     )
#     prediction = response.choices[0].message.content
#     newsData = splitIntoContent(prediction)
#     save_sport(teamName, newsData['title'], newsData['content'])
#     print(f'{teamName} >> {newsData['title']}')



In [251]:
jornal = CorreioBraziliense()
url = 'https://www.correiobraziliense.com.br/esportes/2024/08/6919907-rosario-central-x-fortaleza-onde-assistir-escalacoes-e-arbitragem.html'
newsData = jornal.getNewsData(url, False)
print(newsData)

RosÃ¡rio Central x Fortaleza: onde assistir, escalaÃ§Ãµes e arbitragem
postado em 13/08/2024 22:48                                                    
13/08/2024
{'url': 'https://www.correiobraziliense.com.br/esportes/2024/08/6919907-rosario-central-x-fortaleza-onde-assistir-escalacoes-e-arbitragem.html', 'title': 'RosÃ¡rio Central x Fortaleza: onde assistir, escalaÃ§Ãµes e arbitragem', 'content': 'Nesta quarta-feira (14), o Fortaleza enfrentará o Rosário Central, da Argentina, no primeiro jogo das oitavas de final da Sul-Americana. A partida será realizada no Estádio Gigante de Arroyito. Onde assistir Os canais Disney ESPN vão transmitir a partida. O time, comandado pelo técnico Miguel Ángel Russo chega com algumas dúvidas. Entre elas, se Abel Hernández começará como titular. Contudo, Campaz e Marco Rubén, velhos conhecidos do futebol brasileiro estão confirmados no time argentino O atacante Lucero ainda se recupera de lesão e será desfalque. Assim, Kayser deve ser o titular. O volant